## ShopAssist AI 2.0: Seamless and Cost-Efficient Function Calling API

###Optimized System Design:
Introduces a combined function-calling API for seamless integration, reducing code complexity and improving cost efficiency.

###Redefined Conversation Flow:
Ensures consistent output across layers, minimizing the need for additional processing or manipulation.

###Streamlined Input Handling:
Assumes the input file updated_laptop.csv, where the last column contains Python dictionaries.
Excludes the product_map_layer for simplicity, focusing solely on the function-calling API prototype.

###Testing Requirements:
Ensure the updated_laptop.csv and OpenAI_API_Key.txt files are available in the Google Drive folder Shop Assist under MyDrive.

###Conversation History:
Logs chat history to a text file for future reference and analysis.


In [ ]:
#Install openai
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Shop Assist')
!ls
## Import the necessary libraries for building the project
import os,json,ast
import openai
from openai import OpenAI
import pandas as pd
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored



## Add your OPENAI API KEY
openai.api_key = open("OpenAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

Mounted at /content/drive
laptop_data.csv  OpenAI_API_Key.txt  updated_laptop.csv


In [11]:
import pandas as pd
from IPython.display import display, HTML
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset
df = pd.read_csv('updated_laptop.csv')
df

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,laptop_feature
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"{'GPU intensity': 'medium', 'Display quality':..."
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"{'GPU intensity': 'high', 'Display quality': '..."
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...,"{'GPU intensity': 'low', 'Display quality': 'm..."
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,"{'GPU intensity': 'low', 'Display quality': 'l..."
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,"{'GPU intensity': 'high', 'Display quality': '..."
5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.2 kg,Dual Cooling Fans,1 year,5 hours,"80,000",The Acer Predator is a powerhouse laptop desig...,"{'GPU intensity': 'high', 'Display quality': '..."
6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8GB,SSD,PixelSense,"13.5""",Intel Iris Plus,2256x1504,Windows 11,1.3 kg,Touchscreen Display,1 year,10 hours,"90,000",The Microsoft Surface Laptop is a premium devi...,"{'GPU intensity': 'medium', 'Display quality':..."
7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.6 kg,Backlit Keyboard,3 years,6 hours,"60,000",The Lenovo ThinkPad is a powerful laptop desig...,"{'GPU intensity': 'high', 'Display quality': '..."
8,HP,Pavilion,i5,Intel,2.3 GHz,12GB,HDD,LCD,"15.6""",Intel UHD,1366x768,Windows 10,2.1 kg,B&O Audio,1 year,4 hours,"30,000",The HP Pavilion is a budget-friendly laptop th...,"{'GPU intensity': 'low', 'Display quality': 'l..."
9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16GB,SSD,IPS,"17.3""",NVIDIA RTX,1920x1080,Windows 10,2.9 kg,Aura Sync RGB Keyboard,2 years,5 hours,"85,000",The ASUS ROG Strix G is a high-performance gam...,"{'GPU intensity': 'high', 'Display quality': '..."


###1 - Introducing openAI Function calling API to minimize the manipulations and api calls

In [2]:
## Function Description for the Function Calling API
tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_laptop_recommendations",
                    "description": "Get the top 3 laptops from the catalogue, that best matches what the user is asking based on 'GPU intensity','Display quality','Portability','Multitasking','Processing speed' & 'Budget",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "gpu intensity": {
                                "type": "string",
                                "description": "The requirement of the user in GPU capacity classfied as low, medium or high" ,
                            },
                            "display quality": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's Display Quality & capacity classfied as low, medium or high" ,
                            },
                            "portability": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's portability classfied as low, medium or high" ,
                            },
                            "multitasking": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's Multitasking classfied as low, medium or high" ,
                            },
                            "processing speed": {
                                "type": "string",
                                "description": "The requirement of the user for Laptop's Processing speed classfied as low, medium or high" ,
                            },
                            "budget": {
                                "type": "integer",
                                "description": "The maximum budget of the user" ,
                            },

                        },
                        "required": ["GPU intensity","Display quality","Portability","Multitasking","Processing speed","Budget"],
                    },
                }
            }
        ]

###2 - This initializes the variable conversation with the system message. Uses an improved prompt with 50% reduced tokens than the original prompt with simplified ,streamlined instructions and explicit chain of thougts

In [3]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'GPU intensity': "high",
                        'Display quality':"high",
                        'Portability': "low",
                        'Multitasking': "high",
                        'Processing speed': "high",
                        'Budget': "150000"}


    system_message = f"""
        You are an intelligent laptop gadget expert. Your goal is to find the best laptop for a user by asking relevant questions and understanding their profile. You need to confidently fill the values for the keys ('GPU intensity', 'Display quality', 'Portability', 'Multitasking', 'Processing speed', 'Budget') in the Python dictionary based on the user's responses.
        The Python dictionary should look like this:
        {{'GPU intensity': 'values', 'Display quality': 'values', 'Portability': 'values', 'Multitasking': 'values', 'Processing speed': 'values', 'Budget': 'values'}}
        Instructions for filling the dictionary:
        - The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance stated by the user.
        - The value for 'Budget' should be a numerical value extracted from the user's response and must be greater than or equal to 25000 INR. If the budget is below this, inform the user there are no laptops in that range.
        - Do not randomly assign values. Ensure all values are inferred from the user's responses.
        Chain of Thoughts:
        1. Ask questions to understand the user's profile and requirements. Focus on determining the values for the keys ('GPU intensity', 'Display quality', 'Portability', 'Multitasking', 'Processing speed', 'Budget'). If unclear, ask follow-up questions.
        2. For keys not filled in the previous step, ask specific questions to gather the necessary information. Use logical questions rather than directly citing the key.
        3. Verify if you have correctly filled the values for all keys. If uncertain, ask clarifying questions.
        Example User Interaction:
        User: "Hi, I am an editor."
        Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. You would need high multitasking capability and a high-end display for better visuals. Are you more involved in video editing, photo editing, or both?"
        User: "I primarily work with After Effects."
        Assistant: "Working with After Effects involves graphics, animations, and rendering, requiring a high GPU. Do you work with high-resolution media files, such as 4K videos?"
        User: "Yes, sometimes I work with 4K videos."
        Assistant: "Processing 4K videos requires a good processor and high GPU. Do you need a lightweight laptop for frequent travel or work primarily from a stationary location?"
        User: "I travel but do not carry my laptop."
        Assistant: "Please let me know your budget for the laptop."
        User: "My max budget is 1.5 lakh INR."
        Assistant: "{example_user_dict}"
        Start with a short welcome message and encourage the user to share their requirements.
        """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

###3 - Chat completions api modified to use function calling using tools .

In [4]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))

def get_chat_model_completions(messages,tools=None, tool_choice=None, model="gpt-3.5-turbo"):
    client = OpenAI()
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0, # this is the degree of randomness of the model's output
            max_tokens = 500,
            tools=tools
        )
        response = json.loads(response.to_json())
        # print(response)
        return response["choices"][0]["message"]
    except Exception as e:
        print(f"Exception: {e}")
        return e

###4 - Provide a color coding to each role so that you can differentiate the queries and responses easily in the chatbot

Extract dictionary from string

In [5]:
import pandas as pd
import ast
import re

def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

Moderation_check to validate the input for flags . Reduced lines of code to use if else as part of return

In [6]:
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(user_input):
    client = OpenAI()
    # Call the OpenAI API to perform moderation on the user's input.
    response = client.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    flagged = response.results[0].flagged
    return "Flagged" if flagged else "Not Flagged"

Extract the laptop specs from the descriptions

Added few shot learnings

Compare laptops with user requests

In [32]:
def compare_laptops_with_user(user_req_string):
  budget = user_req_string.get('budget', '0')

  # budget = user_req_string.get('Budget', '0').replace(',', '').split()[0]
  budget = int(budget)
  laptop_df= pd.read_csv('updated_laptop.csv')
  filtered_laptops = laptop_df.copy()
  filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
  filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()
  # print("1")
  mappings = {
          'low': 0,
          'medium': 1,
          'high': 2
      }
  # Create 'Score' column in the DataFrame and initialize to 0
  filtered_laptops['Score'] = 0
  for index, row in filtered_laptops.iterrows():
      user_product_match_str = row['laptop_feature']
      laptop_values = extract_dictionary_from_string(user_product_match_str)
      score = 0

      for key, user_value in user_req_string.items():
        # print(key, user_value)
        if key.lower() == 'budget':
            continue  # Skip budget comparison
        laptop_value = laptop_values.get(key, None)
        laptop_mapping = mappings.get(laptop_value.lower(), -1)
        user_mapping = mappings.get(user_value.lower(), -1)
        if laptop_mapping >= user_mapping:
          ### If the laptop value is greater than or equal to the user value the score is incremented by 1
          score += 1
      filtered_laptops.loc[index, 'Score'] = score

  # Sort the laptops by score in descending order and return the top 5 products
  top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
  top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

  return top_laptops.to_json(orient='records')



Validate the recommendations with score

In [8]:
def recommendation_validation(laptop_recommendation):

    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
      if data[i]['Score'] > 2:
        data1.append(data[i])

    return json.dumps(data1)

###5 - Dialogue Management system with less manipulation and reduced api calls

1.intent_confirmation_layer and dictionary_present replaced by a simple function call to generate the response always as a python dictionary as mentioned in the function call

In [34]:
def dialogue_mgmt_system():
    # Initialize the conversation and create the assistant object instance
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)["content"]

    print(introduction + '\n')

    user_input = ""
    # Step 1: check moderation
    while(user_input != "exit"):
      user_input = input("")
      moderation = moderation_check(user_input)
      if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
      conversation.append({"role": "user", "content": user_input})

      # Step 2: Pass the user input to GPT
      response_assistant = get_chat_model_completions(conversation,tools=tools)

      if "tool_calls" in list(response_assistant.keys()) and response_assistant["tool_calls"]:
        function_called_message = "\nThank you for your inputs. Please wait, while I fetch relevant products\n"
        conversation.append({"role": "assistant", "content": function_called_message})
        print(function_called_message)

        # Step 3: Extract top3 laptops by calling the external function
        tool  = response_assistant["tool_calls"][0]
        tool_call_id = tool["id"]
        function_name = tool["function"]["name"]
        function_args = json.loads(tool["function"]["arguments"])

        top_3_laptops = compare_laptops_with_user(function_args)
        # print('Top 3 laptop',top_3_laptops)
        function_response = recommendation_validation(top_3_laptops)
        # print('function_response',function_response)
        if len(function_response) == 0:
          print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
          break
        # Step 4: send the info on the function call and function response to GPT
        conversation.append(response_assistant)
        conversation.append(
            {
                "tool_call_id": tool_call_id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
        recommendation = get_chat_model_completions(conversation)

        conversation.append({"role": "assistant", "content": recommendation["content"]})
        print("\n" +  recommendation["content"] + "\n")
        print("Enter exit to end the conversation or press enter to continue\n")
      else:
        conversation.append({"role": "assistant", "content": response_assistant["content"]})
        print("\n" +  response_assistant["content"] + "\n")



    # Save conversation in a text file
    with open("conversation.txt", "w") as file:
        for message in conversation:
            file.write(f"{message['role']}: {message['content']}\n")

In [36]:
dialogue_mgmt_system()

Hello! Welcome to the laptop selection assistant. Please share your requirements so that I can help you find the best laptop that suits your needs.

I am a photo editor

Great! As a photo editor, you likely require a laptop with a good display quality for accurate color representation and detail. Do you often work with high-resolution images or RAW files that demand high processing power?

RAW files

Working with RAW files demands a laptop with high processing power to handle the large file sizes efficiently. Do you also need a laptop that can handle multitasking well, especially when working with multiple editing software simultaneously?

No

Understood. Do you prefer a laptop that is highly portable for easy mobility, or is portability not a significant factor for you?

Not a significant factor

Lastly, could you please specify your budget for the laptop? This will help in finding the best options within your price range.

70000

Thank you for your inputs. Please wait, while I fetch 